# Задание по теме "Коллаборативная фильтрация"

Пакет SURPRISE:
- используйте данные MovieLens 1M,
- можно использовать любые модели из пакета,
- получите RMSE на тестовом сете 0,87 и ниже.

In [17]:
import pandas as pd
import numpy as np

from surprise import Dataset, accuracy
from surprise import KNNWithMeans, KNNBasic, SVD
from surprise.model_selection import train_test_split, KFold

## Загрузка данных

In [2]:
data = Dataset.load_builtin("ml-1m")

Dataset ml-1m could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-1m.zip...
Done! Dataset ml-1m has been saved to /home/andrew/.surprise_data/ml-1m


In [7]:
data

## Делим Dataset на обучающую и тестовую выборки

In [8]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=51)

## Обучение моделей

In [14]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:
test_pred = algo.test(testset)

In [16]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9353


0.9352676165031188

## Кроссвалидация

In [21]:
kf = KFold(n_splits=5)

In [22]:
algo = SVD()

In [23]:
for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8727
RMSE: 0.8754
RMSE: 0.8736
RMSE: 0.8713
RMSE: 0.8754
